# File submission workflow for Airflow
Data engineering workflows for analytics commonly require a means to combine manual processing steps, such as generation of data in a Jupyter Notebook or Excel, with automated pipelines. This notebook, together with the associated sample Airflow instance and utility package, demonstrates a way of achieving this, through manually submitting data using an associated utility package which imposes quality requirements before the data is accepted.

## Trying it out
1. run `uv run airflow standalone` in a terminal to start the sample Airflow project
2. log in to the Airflow GUI at http://localhost:8080, using the admin credentials that were printed early in the log
3. open this notebook after first running `uv run --with jupyter jupyter lab` in another terminal

In [1]:
!airflow dags unpause film_files_dag

/home/ed/orchestration-recipes/submit-file-to-airflow/.venv/lib/python3.14/site-packages/airflow/cli/cli_config.py:548 PendingDeprecationWarning: FileType is deprecated. Simply open files after parsing arguments.
/home/ed/orchestration-recipes/submit-file-to-airflow/.venv/lib/python3.14/site-packages/airflow/cli/cli_config.py:735 PendingDeprecationWarning: FileType is deprecated. Simply open files after parsing arguments.
dag_id         | is_paused
===============+==========
film_files_dag | True     
                          


## Inspect utilities
We find a submission function as well as a schema that imposes restrictions on what data we can submit with it.

In [2]:
import pandas as pd
from util import submit_film_file

submit_film_file?

Signature:
submit_film_file(
    df: pandera.typing.pandas.DataFrame[util.FilmSchema],
    filename: str,
)
Docstring: Submit film dataframe to Airflow, if it passes schema validation.
File:      ~/orchestration-recipes/submit-file-to-airflow/util.py
Type:      function

In [3]:
from util import FilmSchema

FilmSchema??

Init signature: FilmSchema(*args, **kwargs) -> pandera.typing.common.DataFrameBase[~TDataFrameModel]
Docstring:     
Model of a pandas :class:`~pandera.api.pandas.container.DataFrameSchema`.

*new in 0.5.0*

See the :ref:`User Guide <dataframe-models>` for more.
Source:        
class FilmSchema(pa.DataFrameModel):
    name: Series[str]
    lead_actor: Series[str]
    rating: Series[int] = pa.Field(ge=0, le=10)
File:           ~/orchestration-recipes/submit-file-to-airflow/util.py
Type:           MetaModel
Subclasses:     

## Try to submit some files
Upon trying to submit our favourite Mads Mikkelsen films, we get rejected 3 times, first because we misspelled a column name, then because we tried to rate something with a float instead of an integer, and finally because we tried to give Druk 11 (while it does deserve it, the schema says 10 is max). Only when all constraints are fulfilled is the submitted data accepted.

Two important features of this workflow are the following.
- **Keeping untidy data out of the orchestrator**: most pipelines will need consistently formatted data to be reliable. The validation step keep inconsistently formatted data out.
- **Instant rejection**: it is frustrating for the submitter to have to wait or investigate whether they made a mistake. This pattern provides the feedback instantly.

In [4]:
try:
    submit_film_file(
        df=pd.DataFrame(
            [
                {"name": "Rogue One: A Star Wars Story", "lead_actorr": "Mads Mikkelsen", "rating": 7},
                {"name": "Bastarden", "lead_actor": "Mads Mikkelsen", "rating": 7.7},
                {"name": "Druk", "lead_actor": "Mads Mikkelsen", "rating": 11},
            ]
        ),
        filename="best_mikkelsen_films.csv",
    )
except Exception as e:
    print(type(e).__name__ + ":")
    print(e)

SchemaError:
non-nullable series 'lead_actor' contains null values:
0    NaN
Name: lead_actor, dtype: object


In [5]:
try:
    submit_film_file(
        df=pd.DataFrame(
            [
                {"name": "Rogue One: A Star Wars Story", "lead_actor": "Mads Mikkelsen", "rating": 7},
                {"name": "Bastarden", "lead_actor": "Mads Mikkelsen", "rating": 7.7},
                {"name": "Druk", "lead_actor": "Mads Mikkelsen", "rating": 11},
            ]
        ),
        filename="best_mikkelsen_films.csv",
    )
except Exception as e:
    print(type(e).__name__ + ":")
    print(e)

SchemaError:
expected series 'rating' to have type int64, got float64


In [6]:
try:
    submit_film_file(
        df=pd.DataFrame(
            [
                {"name": "Rogue One: A Star Wars Story", "lead_actor": "Mads Mikkelsen", "rating": 7},
                {"name": "Bastarden", "lead_actor": "Mads Mikkelsen", "rating": 8},
                {"name": "Druk", "lead_actor": "Mads Mikkelsen", "rating": 11},
            ]
        ),
        filename="best_mikkelsen_films.csv",
    )
except Exception as e:
    print(type(e).__name__ + ":")
    print(e)

SchemaError:
Column 'rating' failed element-wise validator number 1: less_than_or_equal_to(10) failure cases: 11


In [7]:
try:
    submit_film_file(
        df=pd.DataFrame(
            [
                {"name": "Rogue One: A Star Wars Story", "lead_actor": "Mads Mikkelsen", "rating": 7},
                {"name": "Bastarden", "lead_actor": "Mads Mikkelsen", "rating": 8},
                {"name": "Druk", "lead_actor": "Mads Mikkelsen", "rating": 10},
            ]
        ),
        filename="best_mikkelsen_films.csv",
    )
except Exception as e:
    print(type(e).__name__ + ":")
    print(e)

You should now see in the Airflow GUI that a corresponding run has been executed.